## Azure Notebooks 사용하기
- 의외로 다른게 없다..?
- 아니다. 가상환경에서 돌아가기 때문에 프로그래밍 언어 변경/라이브러리 설치,제거 등이 자유롭다.
- 다만 어디까지나 가상환경 기반이므로 장기간 사용하지 않으면 강제로 커널이 초기화될 수는 있다.

### Helloworld.txt의 내용을 보고 싶을 땐 어떤 코드를 짜야할까?

In [ ]:
with open('Helloworld.txt', 'r') as f: #with문을 사용하면 이 코드블럭이 끝날 때 변수 f도 같이 사라진다
    # 나머지 코드 채우기
    